In [1]:
import os

In [2]:
%pwd

'd:\\Brain_MRI_Poject\\MRI_Brain_Tumor_Deployee_AWS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Brain_MRI_Poject\\MRI_Brain_Tumor_Deployee_AWS'

In [48]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [49]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import yaml
from box import ConfigBox

In [50]:
# path_to_yaml = "D:/Classification-Project/config/config.yaml"
path_to_yaml = "D:\Brain_MRI_Poject\MRI_Brain_Tumor_Deployee_AWS\config\config.yaml"

with open(path_to_yaml) as yaml_file:
    content = yaml.safe_load(yaml_file)
    con = ConfigBox(content)


In [51]:
content

{'artifacts_root': 'artifacts',
 'data_ingestion': {'root_dir': 'artifacts/data_ingestion',
  'source_URL': 'https://github.com/Prateek07-Kumar/Brain-in-MRI-Images-for-Brain-Tumor-Detection/blob/main/brain_tumor_dataset.zip',
  'local_data_file': 'artifacts/data_ingestion/data.zip',
  'unzip_dir': 'artifacts/data_ingestion'},
 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model',
  'base_model_path': 'artifacts/prepare_base_model/base_model.h5',
  'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'},
 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks',
  'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir',
  'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'},
 'training': {'root_dir': 'artifacts/training',
  'trained_model_path': 'artifacts/training/model.h5'}}

In [52]:
con

ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Prateek07-Kumar/Brain-in-MRI-Images-for-Brain-Tumor-Detection/blob/main/brain_tumor_dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}})

In [53]:
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)

            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [54]:
# a = read_yaml(r"D:\Brain_MRI_Poject\brain_classification\params.yaml")
                  

a = read_yaml(r"D:\Brain_MRI_Poject\MRI_Brain_Tumor_Deployee_AWS\params.yaml")


In [55]:
a

ConfigBox({'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01})

In [56]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [57]:
import os
os.getcwd()
# os.chdir(r"D:\Brain_MRI_Poject\brain_classification")
os.chdir(r"D:\Brain_MRI_Poject\MRI_Brain_Tumor_Deployee_AWS\config")

In [58]:
os.getcwd()

'D:\\Brain_MRI_Poject\\MRI_Brain_Tumor_Deployee_AWS\\config'

In [59]:
read_yaml(PARAMS_FILE_PATH)

FileNotFoundError: [Errno 2] No such file or directory: 'params.yaml'

In [60]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

In [61]:
CONFIG_FILE_PATH = Path("D:\Brain_MRI_Poject\MRI_Brain_Tumor_Deployee_AWS\config\config.yaml")
PARAMS_FILE_PATH = Path("D:\Brain_MRI_Poject\MRI_Brain_Tumor_Deployee_AWS\params.yaml")

In [62]:
CONFIG_FILE_PATH

WindowsPath('D:/Brain_MRI_Poject/MRI_Brain_Tumor_Deployee_AWS/config/config.yaml')

In [63]:
from box.exceptions import BoxValueError
cont = ConfigurationManager()

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [35]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [36]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [38]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH,PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

: 